Custom code for processing raw data, mapping, and obtaining expression and gene regulation estimates in: <br>
"Evolution of gene expression across brain regions in behaviorally divergent deer mice" <br><br>
Written by Andi Kautt in 2018/2019 <br><br>
I highly recommend installing https://jupyter-contrib-nbextensions.readthedocs.io/en/latest/ and turning the TOC option on <br><br>
This notebook is rather old and very bash heavy; I have since switched to doing pretty much everything in python, so this code looks a bit clunky. But it does get the job done and has very few dependencies

### Set up working environment

#### Import modules

In [ ]:
import sys,os,re,fnmatch

#### Set and create main project dirs and paths

In [ ]:
#create main dirs for project
project_dir = '~/Peromyscus_RNAseq_brain'
data_dir = os.path.join(project_dir,'data')
scripts_dir = os.path.join(project_dir,'scripts')
logs_dir = os.path.join(scripts_dir,'logs')
list_dir = os.path.join(project_dir,'lists')
refseq_dir = os.path.join(project_dir,'refseqs')

for directory in [project_dir,data_dir,scripts_dir,logs_dir,list_dir,refseq_dir]:
    if not os.path.exists(directory):
        os.makedirs(directory, exists_ok=True)

**Download reference sequences and annotations to $refseq_dir and name: <br>
"Pman_2.1.3.fasta", "Ppol_1.3.3.fasta", "Pman_2.1.3.gff3", "Ppol_1.3.3.gff3"**

In [ ]:
# SET RAW DATA PATH and place raw data here in subfolders by region/tissue
raw_data_dir = '' # 

#### Software/tools

In [ ]:
#Picard
picard = '~/software/picard/build/libs'
#cutadapt
#cutadapt 2.3 --> installed with conda in virtual env
#STAR v.2.7.0e
STAR = '~/software/STAR-2.7.0e/bin/Linux_x86_64/STAR'
#RSEM v1.3.1
RSEM_dir = '~/software/RSEM-1.3.1'
#mmseq 1.0.10a
mmseq_dir = '~/software/mmseq-latest/bin'

#### Specify tissues and create dirs

NOTE: "mPFC" was later renamed to "cortex" since since exact definition / precise dissection of medial prefrontal cortex in Peromyscus difficult at the time; referring simply to "cortex" is more conservative

In [ ]:
tissues = ["amygdala","cerebellum","hindbrain","hippocampus","hypothalamus","midbrain","mPFC","septum",
             "striatum","thalamus","wholebrain"]
tissue_file = os.path.join(list_dir,'tissuelist.txt')
if not os.path.exists(tissue_file):
    with open (tissue_file, 'w') as out_tissue_file:
        print(*tissues, sep='\n', file=out_tissue_file)

for tissue in tissues:
    tissue_dir = os.path.join(data_dir,tissue)
    if not os.path.exists(tissue_dir):
        os.mkdir(tissue_dir)

#### Extract sample names

In [ ]:
sample_names = []
tissue_sample_dict = {}

for tissue in tissues:
    samples = []
    tissue_data_path = os.path.join(raw_data_dir,tissue)
    unmapped_bams = [file for file in os.listdir(tissue_data_path) if fnmatch.fnmatch(file,'*.R1.fastq.gz')]
    for sample in unmapped_bams:
        sample = ('_'.join(sample.split('-')[-1].split('.')[0].split('_')[:-1]))
        if not sample in sample_names:
            sample_names.append(sample)
            samples.append(sample)
    tissue_sample_dict[tissue] = samples

    tissue_samplelist = os.path.join(list_dir,tissue + '.samplelist.txt')
    if not os.path.exists(tissue_samplelist):
        with open (tissue_samplelist, 'w') as out_tissue_samplefile:
            print(*tissue_sample_dict[tissue], sep='\n', file=out_tissue_samplefile)

### Trim adapters with CUTADAPT

In [ ]:
%%script env raw_data_dir="$raw_data_dir" data_dir="$data_dir" scripts_dir="$scripts_dir" list_dir="$list_dir" bash

for tissue in $(cat ${list_dir}/tissuelist.txt)

do
    input_dir="$raw_data_dir/${tissue}"
    output_dir="$data_dir/${tissue}"                
    tissue_samplelist="$list_dir/${tissue}.samplelist.txt"

    (
    echo "#!/bin/bash"
    echo "#SBATCH -n 1"
    echo "#SBATCH -c 1"
    echo "#SBATCH -t 0-12:00"
    echo "#SBATCH -p hoekstra,shared,general"
    echo "#SBATCH --mem-per-cpu=100mb"
    echo "#SBATCH --job-name cutadapt"
    echo "#SBATCH --array 1-18%18"
    echo "#SBATCH -o $scripts_dir/logs/%x_%A_%a.out"
    echo "#SBATCH -e $scripts_dir/logs/%x_%A_%a.err"
    echo ""
    printf "module load Anaconda3; source activate py3.6_R3.5 \n"
    echo ""
    printf "sample=\$(awk -v line=\$SLURM_ARRAY_TASK_ID 'NR==line{print \$1; exit}' $tissue_samplelist) \n" 
    echo ""
    printf "if [ ! -f $output_dir/\${sample}.R1.trimmed.fastq.gz ]; then \n \
    cutadapt --cores=1 --minimum-length=25 --quality-cutoff=10 --overlap=3 \
    -a AGATCGGAAGAGC -A AGATCGGAAGAGC --nextseq-trim=10 \
    -o $output_dir/\${sample}.R1.trimmed.fastq.gz -p $output_dir/\${sample}.R2.trimmed.fastq.gz \
    $input_dir/\${sample}.R1.fastq $input_dir/\${sample}.R2.fastq; fi \n"
    ) > ${scripts_dir}/cutadapt.${tissue}.slurm
done

sed -i 's/--array 1-18%18/--array 1-11%11/' $scripts_dir/cutadapt.wholebrain.slurm
sed -i 's/-a AGATCGGAAGAGC -A AGATCGGAAGAGC --nextseq-trim=10/-a AAGATCGGAAGAGC -A GATCGTCGGACTGTAGAA/' $scripts_dir/cutadapt.wholebrain.slurm
sed -i 's/R1.fastq/R1.fastq.gz/; s/R2.fastq/R2.fastq.gz/' $scripts_dir/cutadapt.wholebrain.slurm

In [ ]:
%%script env list_dir="$list_dir" scripts_dir="$scripts_dir" bash

for tissue in $(cat ${list_dir}/tissuelist.txt)

do 
    printf "$tissue -->\t" >> $scripts_dir/logs/jobIDs.cutadapt.log
    sbatch ${scripts_dir}/cutadapt.${tissue}.slurm >> $scripts_dir/logs/jobIDs.cutadapt.log
    sleep 1
done

cat $scripts_dir/logs/jobIDs.cutadapt.log

In [ ]:
%%script env list_dir="$list_dir" scripts_dir="$scripts_dir" bash

sacct -j $(grep -o '[0-9]*' $scripts_dir/logs/jobIDs.cutadapt.log | paste -s -d ",") | grep -v "COMPLETED"

### Create indices (reformat annotations)

#### Add species suffix to fasta (chromosomes) and annotation (chr, ID, transcript_id, gene_id)


In [ ]:
%%script env refseq_dir="$refseq_dir" scripts_dir="$scripts_dir" list_dir="$list_dir" bash

if [ ! -f $refseq_dir/Pman_2.1.3.renamed_suffix.fasta ]; then
    sed -r 's/^>([a-zA-Z0-9]+)/>\1_BW/g' $refseq_dir/Pman_2.1.3.fasta \
    > $refseq_dir/Pman_2.1.3.renamed_suffix.fasta; fi

if [ ! -f $refseq_dir/Ppol_1.3.3.renamed_suffix.fasta ]; then
    sed -r 's/^>([a-zA-Z0-9]+)/>\1_PO/g' $refseq_dir/Ppol_1.3.3.fasta \
    > $refseq_dir/Ppol_1.3.3.renamed_suffix.fasta; fi

     
if [ ! -f $refseq_dir/Pman_2.1.3.renamed_suffix.gff3 ]; then
    sed -r 's/^([a-zA-Z0-9]+)/\1_BW/g' $refseq_dir/Pman_2.1.3.gff3 \
    | sed -r 's/(ID=[a-zA-Z0-9:_+.-]+)/\1_BW/g' \
    | sed -r 's/(transcript_id=[a-zA-Z0-9:_+.-]+)/\1_BW/g' \
    | sed -r 's/(gene_id=[a-zA-Z0-9:_+.-]+)/\1_BW/g' \
    | sed -r 's/(Parent=[a-zA-Z0-9:_+.-]+)/\1_BW/g' \
    | sed -r 's/(Name=[a-zA-Z0-9:_+.-]+)/\1_BW/g' \
    > $refseq_dir/Pman_2.1.3.renamed_suffix.gff3; fi
    
if [ ! -f $refseq_dir/Pman_1.3.3.renamed_suffix.gff3 ]; then
    sed -r 's/^([a-zA-Z0-9]+)/\1_PO/g' $refseq_dir/Ppol_1.3.3.gff3 \
    | sed -r 's/(ID=[a-zA-Z0-9:_+.-]+)/\1_PO/g' \
    | sed -r 's/(transcript_id=[a-zA-Z0-9:_+.-]+)/\1_PO/g' \
    | sed -r 's/(gene_id=[a-zA-Z0-9:_+.-]+)/\1_PO/g' \
    | sed -r 's/(Parent=[a-zA-Z0-9:_+.-]+)/\1_PO/g' \
    | sed -r 's/(Name=[a-zA-Z0-9:_+.-]+)/\1_PO/g' \
    > $refseq_dir/Ppol_1.3.3.renamed_suffix.gff3; fi


# concatenate both to create diploid fasta and annotation for F1s

if [ ! -f $refseq_dir/Pman_2.1.3_Ppol_1.3.3.renamed_suffix.diploid.fasta ]; then
    cat $refseq_dir/Pman_2.1.3.renamed_suffix.fasta \
    $refseq_dir/Ppol_1.3.3.renamed_suffix.fasta \
    > $refseq_dir/Pman_2.1.3_Ppol_1.3.3.renamed_suffix.diploid.fasta; fi
    
if [ ! -f $refseq_dir/Pman_2.1.3_Ppol_1.3.3.renamed_suffix.diploid.gff3 ]; then
    cat $refseq_dir/Pman_2.1.3.renamed_suffix.gff3 \
    <(grep -v "#" $refseq_dir/Ppol_1.3.3.renamed_suffix.gff3) \
    > $refseq_dir/Pman_2.1.3_Ppol_1.3.3.renamed_suffix.diploid.gff3; fi

#### Create RSEM indices

In [ ]:
%%script env refseq_dir="$refseq_dir" scripts_dir="$scripts_dir" list_dir="$list_dir" RSEM_dir="$RSEM_dir" bash

if [ ! -d $refseq_dir/Pman_2.1.3.RSEM_gff3 ]; then
    mkdir $refseq_dir/Pman_2.1.3.RSEM_gff3
    mkdir $refseq_dir/Ppol_1.3.3.RSEM_gff3
    mkdir $refseq_dir/Pman_2.1.3_Ppol_1.3.3.RSEM_gff3; fi

if [ ! -f $refseq_dir/Pman_2.1.3.RSEM_gff3/Pman_2.1.3.RSEM_gff3.n2g.idx.fa ]; then
    sbatch --mem=2gb --time=0-02:00 -p hoekstra,shared,general \
    --job-name "RSEM_index" -o $scripts_dir/logs/RSEM_index.BW.out -e $scripts_dir/logs/RSEM_index.BW.err \
    --wrap="$RSEM_dir/rsem-prepare-reference \
    --gff3-RNA-patterns \"mRNA,transcript,pseudogenic_transcript\" \
    -gff3 $refseq_dir/Pman_2.1.3.gff3 $refseq_dir/Pman_2.1.3.fasta \
    $refseq_dir/Pman_2.1.3.RSEM_gff3/Pman_2.1.3.RSEM_gff3"; fi
        
if [ ! -f $refseq_dir/Ppol_1.3.3.RSEM_gff3/Ppol_1.3.3.RSEM_gff3.n2g.idx.fa ]; then
    sbatch --mem=2gb --time=0-02:00 -p hoekstra,shared,general \
    --job-name "RSEM_index" -o $scripts_dir/logs/RSEM_index.PO.out -e $scripts_dir/logs/RSEM_index.PO.err \
    --wrap="$RSEM_dir/rsem-prepare-reference \
    --gff3-RNA-patterns \"mRNA,transcript,pseudogenic_transcript\" \
    -gff3 $refseq_dir/Ppol_1.3.3.gff3 $refseq_dir/Ppol_1.3.3.fasta \
    $refseq_dir/Ppol_1.3.3.RSEM_gff3/Ppol_1.3.3.RSEM_gff3"; fi
        
if [ ! -f $refseq_dir/Pman_2.1.3_Ppol_1.3.3.RSEM_gff3/Pman_2.1.3_Ppol_1.3.3.RSEM_gff3.n2g.idx.fa ]; then
    sbatch --mem=2gb --time=0-02:00 -p hoekstra,shared,general \
    --job-name "RSEM_index" -o $scripts_dir/logs/RSEM_index.F1.out -e $scripts_dir/logs/RSEM_index.F1.err \
    --wrap="$RSEM_dir/rsem-prepare-reference \
    --gff3-RNA-patterns \"mRNA,transcript,pseudogenic_transcript\" \
    -gff3 $refseq_dir/Pman_2.1.3_Ppol_1.3.3.renamed_suffix.diploid.gff3 $refseq_dir/Pman_2.1.3_Ppol_1.3.3.renamed_suffix.diploid.fasta \
    $refseq_dir/Pman_2.1.3_Ppol_1.3.3.RSEM_gff3/Pman_2.1.3_Ppol_1.3.3.RSEM_gff3"; fi

#### Create STAR indices

In [ ]:
%%script env scripts_dir="$scripts_dir" list_dir="$list_dir" refseq_dir="$refseq_dir" STAR="$STAR" bash

if [ ! -d $refseq_dir/Pman_2.1.3.STAR2.7.0e_gtf ]; then
    mkdir $refseq_dir/Pman_2.1.3.STAR2.7.0e_gtf
    mkdir $refseq_dir/Ppol_1.3.3.STAR2.7.0e_gtf
    mkdir $refseq_dir/Pman_2.1.3_Ppol_1.3.3.diploid.STAR2.7.0e_gtf; fi
    
if [ ! -f $refseq_dir/Pman_2.1.3.STAR2.7.0e_gtf/Genome ]; then
    sbatch --mem=40gb -n 16 --time=0-12:00 -p hoekstra,shared,general \
    --job-name "STAR_index" -o $scripts_dir/logs/STAR_index.BW.out -e $scripts_dir/logs/STAR_index.BW.err \
    --wrap="$STAR --runThreadN 16 --runMode genomeGenerate \
    --genomeDir $refseq_dir/Pman_2.1.3.STAR2.7.0e_gtf \
    --genomeFastaFiles $refseq_dir/Pman_2.1.3.fasta \
    --sjdbGTFfile $refseq_dir/Pman_2.1.3.RSEM_gff3/Pman_2.1.3.RSEM_gff3.gtf \
    --genomeChrBinNbits 17 --sjdbOverhang 149"; fi

if [ ! -f $refseq_dir/Ppol_1.3.3.STAR2.7.0e_gtf/Genome ]; then
    sbatch --mem=40gb -n 16 --time=0-12:00 -p hoekstra,shared,general \
    --job-name "STAR_index" -o $scripts_dir/logs/STAR_index.PO.out -e $scripts_dir/logs/STAR_index.PO.err \
    --wrap="$STAR --runThreadN 16 --runMode genomeGenerate \
    --genomeDir $refseq_dir/Ppol_1.3.3.STAR2.7.0e_gtf \
    --genomeFastaFiles $refseq_dir/Ppol_1.3.3.fasta \
    --sjdbGTFfile $refseq_dir/Ppol_1.3.3.RSEM_gff3/Ppol_1.3.3.RSEM_gff3.gtf \
    --genomeChrBinNbits 17 --sjdbOverhang 149"; fi
        
if [ ! -f $refseq_dir/Pman_2.1.3_Ppol_1.3.3.diploid.STAR2.7.0e_gtf/Genome ]; then
    sbatch --mem=80gb -n 16 --time=0-12:00 -p hoekstra,shared,general \
    --job-name "STAR_index" -o $scripts_dir/logs/STAR_index.F1.out -e $scripts_dir/logs/STAR_index.F1.err \
    --wrap="$STAR --runThreadN 16 --runMode genomeGenerate \
    --genomeDir $refseq_dir/Pman_2.1.3_Ppol_1.3.3.diploid.STAR2.7.0e_gtf \
    --genomeFastaFiles $refseq_dir/Pman_2.1.3_Ppol_1.3.3.renamed_suffix.diploid.fasta \
    --sjdbGTFfile $refseq_dir/Pman_2.1.3_Ppol_1.3.3.RSEM_gff3/Pman_2.1.3_Ppol_1.3.3.RSEM_gff3.gtf \
    --genomeChrBinNbits 17 --sjdbOverhang 149 --limitGenomeGenerateRAM 35000000000"; fi

#### Format transcript fastas for MMSEQ

NOTES:
- Merge ID and name fields to retain both for subsequent manipulations/analyses
- gffread seems to only accept "Name" as geneID field
- MMSEQ expects header to be ">TID ... gene:GID ..." (but could also use regex within)

In [ ]:
%%script env scripts_dir="$scripts_dir" list_dir="$list_dir" refseq_dir="$refseq_dir"  bash

if [ ! -f  $refseq_dir/Pman_2.1.3.RSEM_gff3.renamed.gtf ]; then
    sed 's/ //g; s/;/\t/g; s/"//g' $refseq_dir/Pman_2.1.3.RSEM_gff3/Pman_2.1.3.RSEM_gff3.gtf \
    | awk -F "\t" 'BEGIN{OFS=""} \
    {for (i=9;i<=NF;i++) if ($i~"gene_id") {gsub("gene_id","",$i); gene_id=$i}; \
    for (i=9;i<=NF;i++) if ($i~"transcript_id") {gsub("transcript_id","",$i); transcript_id=$i}; \
    for (i=9;i<=NF;i++) if ($i~"gene_name") {gsub("gene_name","",$i); gene_name=$i}; \
    print $1,"\t",$2,"\t",$3,"\t",$4,"\t",$5,"\t",$6,"\t",$7,"\t",$8,"\t",\
        "Name \"",gene_id,"_",gene_name,"\"","; ","transcript_id \"",transcript_id,"\""}' \
    > $refseq_dir/Pman_2.1.3.RSEM_gff3.renamed.gtf; fi
    
if [ ! -f $refseq_dir/Pman_2.1.3.transcripts.mmseq.fasta ]; then
    gffread -F -w $refseq_dir/Pman_2.1.3.transcripts.mmseq.fasta \
    -g $refseq_dir/Pman_2.1.3.fasta \
    $refseq_dir/Pman_2.1.3.RSEM_gff3.renamed.gtf
    
    sed -i 's/Name=/gene:/' $refseq_dir/Pman_2.1.3.transcripts.mmseq.fasta; fi
 
 
if [ ! -f  $refseq_dir/Ppol_1.3.3.RSEM_gff3.renamed.gtf ]; then
    sed 's/ //g; s/;/\t/g; s/"//g' $refseq_dir/Ppol_1.3.3.RSEM_gff3/Ppol_1.3.3.RSEM_gff3.gtf \
    | awk -F "\t" 'BEGIN{OFS=""} \
    {for (i=9;i<=NF;i++) if ($i~"gene_id") {gsub("gene_id","",$i); gene_id=$i}; \
    for (i=9;i<=NF;i++) if ($i~"transcript_id") {gsub("transcript_id","",$i); transcript_id=$i}; \
    for (i=9;i<=NF;i++) if ($i~"gene_name") {gsub("gene_name","",$i); gene_name=$i}; \
    print $1,"\t",$2,"\t",$3,"\t",$4,"\t",$5,"\t",$6,"\t",$7,"\t",$8,"\t",\
        "Name \"",gene_id,"_",gene_name,"\"","; ","transcript_id \"",transcript_id,"\""}' \
    > $refseq_dir/Ppol_1.3.3.RSEM_gff3.renamed.gtf; fi
    
if [ ! -f $refseq_dir/Ppol_1.3.3.transcripts.mmseq.fasta ]; then
    gffread -F -w $refseq_dir/Ppol_1.3.3.transcripts.mmseq.fasta \
    -g $refseq_dir/Ppol_1.3.3.fasta \
    $refseq_dir/Ppol_1.3.3.RSEM_gff3.renamed.gtf
    
    sed -i 's/Name=/gene:/' $refseq_dir/Ppol_1.3.3.transcripts.mmseq.fasta; fi

    
if [ ! -f  $refseq_dir/Pman_2.1.3_Ppol_1.3.3.RSEM_gff3.renamed.gtf ]; then
    sed 's/ //g; s/;/\t/g; s/"//g' $refseq_dir/Pman_2.1.3_Ppol_1.3.3.RSEM_gff3/Pman_2.1.3_Ppol_1.3.3.RSEM_gff3.gtf \
    | awk -F "\t" 'BEGIN{OFS=""} \
    {for (i=9;i<=NF;i++) if ($i~"gene_id") {gsub("gene_id|_BW|_PO","",$i); gene_id=$i}; \
    for (i=9;i<=NF;i++) if ($i~"transcript_id") {gsub("transcript_id","",$i); transcript_id=$i}; \
    for (i=9;i<=NF;i++) if ($i~"gene_name") {gsub("gene_name","",$i); gene_name=$i}; \
    print $1,"\t",$2,"\t",$3,"\t",$4,"\t",$5,"\t",$6,"\t",$7,"\t",$8,"\t",\
        "Name \"",gene_id,"_",gene_name,"\"","; ","transcript_id \"",transcript_id,"\""}' \
    > $refseq_dir/Pman_2.1.3_Ppol_1.3.3.RSEM_gff3.renamed.gtf; fi
    
if [ ! -f $refseq_dir/Pman_2.1.3_Ppol_1.3.3.renamed_suffix.diploid.transcripts.mmseq.fasta ]; then
    gffread -F -w $refseq_dir/Pman_2.1.3_Ppol_1.3.3.renamed_suffix.diploid.transcripts.mmseq.fasta \
    -g $refseq_dir/Pman_2.1.3_Ppol_1.3.3.renamed_suffix.diploid.fasta \
    $refseq_dir/Pman_2.1.3_Ppol_1.3.3.RSEM_gff3.renamed.gtf
    
    sed -i 's/Name=/gene:/' $refseq_dir/Pman_2.1.3_Ppol_1.3.3.renamed_suffix.diploid.transcripts.mmseq.fasta; fi

#### Create genelist bed files

In [ ]:
%%script env scripts_dir="$scripts_dir" list_dir="$list_dir" refseq_dir="$refseq_dir"  bash

if [ ! $list_dir/Pman_2.1.3.gene_name.genelist.bed ]; then
    sed 's/;/\t/g' $refseq_dir/Pman_2.1.3.gff3 \
    | awk -F "\t" 'BEGIN{OFS="\t"} $3=="gene" || $3=="pseudogene" \
    {for (i=9;i<=NF;i++) if ($i~"Name") {gsub("Name=","",$i); gene_id=$i}; \
    print $1,$4-1,$5,$3,$2,gene_id}' \
     > $list_dir/Pman_2.1.3.gene_name.genelist.bed; fi
    
if [ ! $list_dir/Ppol_1.3.3.gene_name.genelist.bed ]; then
    sed 's/;/\t/g' $refseq_dir/Ppol_1.3.3.gff3 \
    | awk -F "\t" 'BEGIN{OFS="\t"} $3=="gene" || $3=="pseudogene" \
    {for (i=9;i<=NF;i++) if ($i~"Name") {gsub("Name=","",$i); gene_id=$i}; \
    print $1,$4-1,$5,$3,$2,gene_id}' \
    > $list_dir/Ppol_1.3.3.gene_name.genelist.bed; fi 
    
if [ ! $list_dir/Pman_2.1.3_Ppol_1.3.3.gene_name.chrX.genelist.txt ]; then
    cat $list_dir/Pman_2.1.3.gene_name.genelist.bed $list_dir/Ppol_1.3.3.gene_name.genelist.bed \
    | grep chrX | cut -f 6 | sort | uniq \
    > $list_dir/Pman_2.1.3_Ppol_1.3.3.gene_name.chrX.genelist.txt; fi

### Map to genome with STAR

In [ ]:
%%script env data_dir="$data_dir" scripts_dir="$scripts_dir" list_dir="$list_dir" refseq_dir="$refseq_dir" STAR="$STAR" bash

BW_refseq="$refseq_dir/Pman_2.1.3.STAR2.7.0e_gtf"
PO_refseq="$refseq_dir/Ppol_1.3.3.STAR2.7.0e_gtf"
F1_refseq="$refseq_dir/Pman_2.1.3_Ppol_1.3.3.diploid.STAR2.7.0e_gtf"

for tissue in $(cat ${list_dir}/tissuelist.txt)

do
    input_dir="$data_dir/${tissue}"
    output_dir="$data_dir/${tissue}"                
    tissue_samplelist="$list_dir/${tissue}.samplelist.txt"

    (
    echo "#!/bin/bash"
    echo "#SBATCH -n 16"
    echo "#SBATCH -N 1"
    echo "#SBATCH -t 0-04:00"
    echo "#SBATCH -p hoekstra,shared,general"
    echo "#SBATCH --mem-per-cpu=5gb"
    echo "#SBATCH --job-name STAR"
    echo "#SBATCH --array 1-18%18"
    echo "#SBATCH -o $scripts_dir/logs/%x_%A_%a.out"
    echo "#SBATCH -e $scripts_dir/logs/%x_%A_%a.err"
    echo ""
    printf "sample=\$(awk -v line=\$SLURM_ARRAY_TASK_ID 'NR==line{print \$1; exit}' $tissue_samplelist) \n" 
    echo ""
    printf "case \$sample in \n\
    BW*) refseq=\"$BW_refseq\" ;; \n\
    PO*) refseq=\"$PO_refseq\" ;; \n\
    F1*) refseq=\"$F1_refseq\" ;; \n\
    *) echo \"Sample name does not contain pop info. Refseq could not be specified.\" ;; \n\
    esac \n"
    echo ""
    printf "if [ ! -f $output_dir/\${sample}.STAR.Aligned.out.bam ]; then \n \
    $STAR --runThreadN 16 --genomeDir \$refseq --readFilesIn \
    $input_dir/\${sample}.R1.trimmed.fastq.gz $input_dir/\${sample}.R2.trimmed.fastq.gz \
    --outFileNamePrefix $output_dir/\${sample}.STAR. --runMode alignReads \
    --readFilesCommand zcat --twopassMode Basic --outFilterType BySJout --outFilterMultimapNmax 20 \
    --outFilterMismatchNmax 999 --outFilterMismatchNoverLmax 0.04 --alignIntronMin 20 --alignIntronMax 1000000 \
    --alignMatesGapMax 1000000 --alignSJoverhangMin 8 --alignSJDBoverhangMin 1 -sjdbScore 1 --genomeLoad NoSharedMemory \
    --outSAMattributes All --outSAMtype BAM Unsorted --quantMode TranscriptomeSAM --quantTranscriptomeBan \
    IndelSoftclipSingleend --outSAMattrRGline ID:\${sample} PL:illumina LB:\${sample} SM:\${sample} PU:barcode; fi \n"
    ) > ${scripts_dir}/STAR.${tissue}.slurm

done

sed -i 's/--array 1-18%18/--array 1-11%11/' $scripts_dir/STAR.wholebrain.slurm

In [ ]:
%%script env list_dir="$list_dir" scripts_dir="$scripts_dir" bash

for tissue in $(cat ${list_dir}/tissuelist.txt)

do 
    printf "$tissue -->\t" >> $scripts_dir/logs/jobIDs.STAR.log
    sbatch ${scripts_dir}/STAR.${tissue}.slurm >> $scripts_dir/logs/jobIDs.STAR.log
    sleep 1
done

cat $scripts_dir/logs/jobIDs.STAR.log

In [ ]:
%%script env list_dir="$list_dir" scripts_dir="$scripts_dir" bash

sacct -j $(grep -o '[0-9]*' $scripts_dir/logs/jobIDs.STAR.log | paste -s -d ",") | grep -v "COMP"

In [ ]:
%%script env data_dir="$data_dir" scripts_dir="$scripts_dir" list_dir="$list_dir" bash

samtools quickcheck -v ${data_dir}/*/*.bam > $list_dir/bad_bams.fofn && echo 'all bam files ok' \
        || echo 'some files failed check, see bad_bams.fofn'

#### Summarize mapping stats

In [ ]:
%%script env data_dir="$data_dir" list_dir="$list_dir" scripts_dir="$scripts_dir" bash

output_dir="$data_dir/mapping_stats"

if [ ! -d $output_dir ]; then mkdir $output_dir; fi


for tissue in $(cat ${list_dir}/tissuelist.txt)
do
    if [ -f $output_dir/$tissue.STAR.stats.txt ]; then rm $output_dir/$tissue.STAR.stats.txt; fi
        
    sample=$(head -1 $list_dir/${tissue}.samplelist.txt)
    awk 'BEGIN{OFS="\t"}{a[NR]=$1} \
        END{for(i=1;i<=NR;i++){str=str"\t"a[i]; } print "Sample",str}' \
        <(tail -n +6 $data_dir/$tissue/${sample}.STAR.Log.final.out \
          | tr -d " " | tr -d "|" | sed '/^\s*$/d' | grep -v "READS") \
        > $output_dir/$tissue.STAR.stats.txt
         
    for sample in $(cat $list_dir/${tissue}.samplelist.txt)
    do
        awk -v sample=$sample 'BEGIN{OFS="\t"}{a[NR]=$2} \
            END{for(i=1;i<=NR;i++){str=str"\t"a[i]; } print sample,str}' \
            <(tail -n +6 $data_dir/$tissue/${sample}.STAR.Log.final.out \
              | tr -d " " | tr -d "|" | sed '/^\s*$/d' | grep -v "READS") \
            >> $output_dir/$tissue.STAR.stats.txt
    done
done

### Estimate expression with RSEM

In [ ]:
%%script env data_dir="$data_dir" scripts_dir="$scripts_dir" refseq_dir="$refseq_dir" list_dir="$list_dir" RSEM_dir="$RSEM_dir" bash

BW_RSEM_index="$refseq_dir/Pman_2.1.3.RSEM_gff3/Pman_2.1.3.RSEM_gff3"
PO_RSEM_index="$refseq_dir/Ppol_1.3.3.RSEM_gff3/Ppol_1.3.3.RSEM_gff3"
F1_RSEM_index="$refseq_dir/Pman_2.1.3_Ppol_1.3.3.RSEM_gff3/Pman_2.1.3_Ppol_1.3.3.RSEM_gff3"

for tissue in $(cat ${list_dir}/tissuelist.txt)

do
    work_dir="$data_dir/${tissue}"                
    tissue_samplelist="$list_dir/${tissue}.samplelist.txt"

    (
    echo "#!/bin/bash"
    echo "#SBATCH -n 1"
    echo "#SBATCH -c 16"   
    echo "#SBATCH -N 1"
    echo "#SBATCH -t 0-08:00"
    echo "#SBATCH -p hoekstra,shared,general"
    echo "#SBATCH --mem=8gb"
    echo "#SBATCH --job-name RSEM"
    echo "#SBATCH --array 1-18%18"
    echo "#SBATCH -o $scripts_dir/logs/%x_%A_%a.out"
    echo "#SBATCH -e $scripts_dir/logs/%x_%A_%a.err"
    echo ""
    printf "sample=\$(awk -v line=\$SLURM_ARRAY_TASK_ID 'NR==line{print \$1; exit}' $tissue_samplelist) \n" 
    echo ""
    printf "case \$sample in \n\
    BW*) RSEM_index=\"$BW_RSEM_index\" ;; \n\
    PO*) RSEM_index=\"$PO_RSEM_index\" ;; \n\
    F1*) RSEM_index=\"$F1_RSEM_index\" ;; \n\
    *) echo \"Sample name does not contain pop info. RSEM_index could not be specified.\" ;; \n\
    esac \n"
    echo ""
    printf "if [ ! -f $work_dir/\${sample}.RSEM.genes.results ]; then \n \
    $RSEM_dir/rsem-calculate-expression --num-threads 16 --strandedness reverse --paired-end --alignments \
    --append-names --no-bam-output --time \
    $work_dir/\${sample}.STAR.Aligned.toTranscriptome.out.bam \$RSEM_index \
    $work_dir/\${sample}.RSEM; fi \n"
    ) > ${scripts_dir}/RSEM.${tissue}.slurm

done

sed -i 's/--array 1-18%18/--array 1-11%11/' $scripts_dir/RSEM.wholebrain.slurm
sed -i 's/--strandedness reverse //' $scripts_dir/RSEM.wholebrain.slurm

In [ ]:
%%script env list_dir="$list_dir" scripts_dir="$scripts_dir" bash

for tissue in $(cat ${list_dir}/tissuelist.txt)

do 
    printf "$tissue -->\t" >> $scripts_dir/logs/jobIDs.RSEM.log
    sbatch ${scripts_dir}/RSEM.${tissue}.slurm >> $scripts_dir/logs/jobIDs.RSEM.log
    sleep 1
done

cat $scripts_dir/logs/jobIDs.RSEM.log

In [ ]:
%%script env list_dir="$list_dir" scripts_dir="$scripts_dir" bash

sacct -j $(grep -o '[0-9]*' $scripts_dir/logs/jobIDs.RSEM.log | paste -s -d ",") | grep -v "COMP"

#### Create count tables

##### Create list of shared gene_names

In [ ]:
%%script env data_dir="$data_dir" scripts_dir="$scripts_dir" list_dir="$list_dir" refseq_dir="$refseq_dir" bash

# only keep genes that have gene_name based on orthology with Mus

if [ ! -f $list_dir/BW_PO.shared_genes_byName.genelist.txt ]; then 
    awk 'FNR==NR && !/APOLLO/ && !/None/ {gsub("DEERMOUSE_G[0-9]+_","",$1); idx[$1]=$1; next} \
    FNR!=NR && !/APOLLO/ && !/None/ {gsub("OLDFIELD_G[0-9]+_","",$1); if ($1 in idx) print idx[$1]}' \
    $data_dir/amygdala/BW_M_15507_amygdala.RSEM.genes.results \
    $data_dir/amygdala/PO_F_15818_amygdala.RSEM.genes.results \
    > $list_dir/BW_PO.shared_genes_byName.genelist.txt; fi
    
wc -l $list_dir/BW_PO.shared_genes_byName.genelist.txt && head $list_dir/BW_PO.shared_genes_byName.genelist.txt

In [ ]:
%%script env scripts_dir="$scripts_dir" list_dir="$list_dir" refseq_dir="$refseq_dir" data_dir="$data_dir" bash

if [ ! $list_dir/BW_PO.shared_genes_byName_bySourceGene.genelist.txt ]; then
    sed 's/;/\t/g' $refseq_dir/Pman_2.1.3.gff3 \
    | awk -F "\t" 'BEGIN{OFS="\t"} $3=="gene" || $3=="pseudogene" \
    {for (i=9;i<=NF;i++) if ($i~"Name") {gsub("Name=","",$i); gene_id=$i; \
    for (i=9;i<=NF;i++) if ($i~"source_gene") {gsub("source_gene=","",$i); source_gene_id=$i}}; \
    print gene_id,source_gene_id}' \
    | awk 'BEGIN{OFS="\t"} FNR==NR {idx[$1]=$0} \
    FNR!=NR {if ($1 in idx) print idx[$1]}' \
    - $list_dir/BW_PO.shared_genes_byName.genelist.txt \
    > $list_dir/BW_PO.shared_genes_byName_bySourceGene.genelist.txt; fi
    
wc -l $list_dir/BW_PO.shared_genes_byName_bySourceGene.genelist.txt && head $list_dir/BW_PO.shared_genes_byName_bySourceGene.genelist.txt

##### Extract shared genes in parental species

In [ ]:
%%script env data_dir="$data_dir" scripts_dir="$scripts_dir" list_dir="$list_dir" bash

for tissue in $(cat $list_dir/tissuelist.txt)
do
    for sample in $(grep "BW\|PO" $list_dir/${tissue}.samplelist.txt)
    do

    if [ ! -f $data_dir/$tissue/${sample}.RSEM.genes.shared.results ]; then 
        awk 'BEGIN{OFS="\t"} FNR==NR && !/APOLLO/ && !/None/ \
        {gsub("DEERMOUSE_G[0-9]+_|OLDFIELD_G[0-9]+_","",$1); idx[$1]=$0} \
        FNR!=NR {if ($1 in idx) print idx[$1]}' \
        $data_dir/$tissue/${sample}.RSEM.genes.results \
        $list_dir/BW_PO.shared_genes_byName.genelist.txt \
        > $data_dir/$tissue/${sample}.RSEM.genes.shared.results; fi
    done
done

##### Get sum of both alleles in F1s

In [ ]:
%%script env data_dir="$data_dir" scripts_dir="$scripts_dir" list_dir="$list_dir" bash

for tissue in $(cat $list_dir/tissuelist.txt)
do
    for sample in $(grep "F1" $list_dir/${tissue}.samplelist.txt)
    do
        if [ ! -f $data_dir/$tissue/${sample}.RSEM.genes.shared.sum.results ]; then
            awk 'BEGIN{OFS="\t"} NR>1 && !/APOLLO/ && !/None/ \
            {gsub("DEERMOUSE_G[0-9]+_|OLDFIELD_G[0-9]+_|_BW|_PO|BW_|PO_","",$1); sum[$1]+=$5} \
            END{print "gene_id","transcript_id(s)","length","effective_length","expected_count","TPM","FPKM"; \
            for (i in sum) print i,"NA","NA","NA",sum[i],"NA","NA"}' \
            $data_dir/$tissue/${sample}.RSEM.genes.results |
            awk 'BEGIN{OFS="\t"} FNR==NR {idx[$1]=$0; next} \
            FNR!=NR {if ($1 in idx) print idx[$1]}' \
            - $list_dir/BW_PO.shared_genes_byName.genelist.txt \
            > $data_dir/$tissue/${sample}.RSEM.genes.shared.sum.results; fi
    done
done

##### Get counts for parental alleles in F1s separately

In [ ]:
%%script env data_dir="$data_dir" scripts_dir="$scripts_dir" list_dir="$list_dir" bash

for tissue in $(cat $list_dir/tissuelist.txt)
do
    for sample in $(grep "F1" $list_dir/${tissue}.samplelist.txt)
    do
        if [ ! -f $data_dir/$tissue/${sample}.RSEM.genes.shared.BW.results ]; then
            awk 'BEGIN{OFS="\t"} NR==1 {print} FNR==NR && /BW/ && !/APOLLO/ && !/None/ \
            {gsub("DEERMOUSE_G[0-9]+_|OLDFIELD_G[0-9]+_|_BW|_PO|BW_|PO_","",$1); \
            gsub("_BW|_PO|BW_|PO_","",$2); idx[$1]=$0} \
            FNR!=NR {if ($1 in idx) print idx[$1]}' \
            $data_dir/$tissue/${sample}.RSEM.genes.results \
            $list_dir/BW_PO.shared_genes_byName.genelist.txt \
            > $data_dir/$tissue/${sample}.RSEM.genes.shared.BW.results; fi
    
        if [ ! -f $data_dir/$tissue/${sample}.RSEM.genes.shared.PO.results ]; then
            awk 'BEGIN{OFS="\t"} NR==1 {print} FNR==NR && /PO/ && !/APOLLO/ && !/None/ \
            {gsub("DEERMOUSE_G[0-9]+_|OLDFIELD_G[0-9]+_|_BW|_PO|BW_|PO_","",$1); \
            gsub("_BW|_PO|BW_|PO_","",$2); idx[$1]=$0} \
            FNR!=NR {if ($1 in idx) print idx[$1]}' \
            $data_dir/$tissue/${sample}.RSEM.genes.results \
            $list_dir/BW_PO.shared_genes_byName.genelist.txt \
            > $data_dir/$tissue/${sample}.RSEM.genes.shared.PO.results; fi
    done
done

##### Generate count tables

In [ ]:
%%script env data_dir="$data_dir" scripts_dir="$scripts_dir" list_dir="$list_dir" RSEM_dir="$RSEM_dir" bash

### all tissues - with sum of both alleles in F1s

output_dir="$data_dir/RSEM_count_tables"

if [ ! -f $output_dir/BW_PO_F1sum.all_subregions_WB.genes.shared.count.matrix ]; then

    samples=""
    
    for tissue in $(grep -v "wholebrain" $list_dir/tissuelist.txt)
    do
        samples+=$(find $data_dir/$tissue/BW*.genes.shared.results); samples+=" "
        samples+=$(find $data_dir/$tissue/PO*.genes.shared.results); samples+=" "
        samples+=$(find $data_dir/$tissue/F1*.genes.shared.sum.results); samples+=" "
    done
    
    samples+=$(find $data_dir/wholebrain/BW*.genes.shared.results); samples+=" "
    samples+=$(find $data_dir/wholebrain/PO*.genes.shared.results); samples+=" "

    $RSEM_dir/rsem-generate-data-matrix $samples \
        | sed -r "s,$data_dir/[a-zA-Z_]+/,,g" | sed -r 's,\.RSEM\.genes(\.shared)?(\.sum)?\.results,,g' \
        > $output_dir/BW_PO_F1sum.all_subregions_WB.genes.shared.count.matrix; fi

In [ ]:
%%script env data_dir="$data_dir" scripts_dir="$scripts_dir" list_dir="$list_dir" RSEM_dir="$RSEM_dir" bash

### INFO FILE - all tissues - with F1s sum of both alleles 

output_dir="$data_dir/RSEM_count_tables"

if [ ! -f $output_dir/BW_PO_F1sum.all_subregions_WB.genes.shared.info.txt ]; then

    samples=""
    
    for tissue in $(grep -v "wholebrain" $list_dir/tissuelist.txt)
    do
        samples+=$(find $data_dir/$tissue/BW*.genes.shared.results -exec basename {} \;); samples+=" "
        samples+=$(find $data_dir/$tissue/PO*.genes.shared.results -exec basename {} \;); samples+=" "
        samples+=$(find $data_dir/$tissue/F1*.genes.shared.sum.results -exec basename {} \;); samples+=" "
    done
    
    samples+=$(find $data_dir/wholebrain/BW*.genes.shared.results -exec basename {} \;); samples+=" "
    samples+=$(find $data_dir/wholebrain/PO*.genes.shared.results -exec basename {} \;); samples+=" "
    

    echo $samples | tr " " "\n" | sed -r 's,\..+,,g' \
        | awk 'BEGIN{OFS="\t"} {split($0,a,"_"); print $1,a[1],a[2],a[4]}' \
        | awk 'BEGIN{OFS="\t"; print "sample_id\tspecies\tsex\ttissue\tbatch"} \
        /cerebellum|hindbrain|hippocampus|midbrain|_thalamus/ {print $0,"1"} \
        /amygdala|hypothalamus|mPFC|septum|striatum/ {print $0,"2"} \
        /wholebrain/ {print $0,"3"}' \
        > $output_dir/BW_PO_F1sum.all_subregions_WB.genes.shared.info.txt; fi

### Estimate expression with MMSEQ

#### bam2hits

In [ ]:
%%script env data_dir="$data_dir" scripts_dir="$scripts_dir" refseq_dir="$refseq_dir" list_dir="$list_dir" mmseq_dir="$mmseq_dir" bash

BW_refseq="$refseq_dir/Pman_2.1.3.transcripts.mmseq.fasta"
PO_refseq="$refseq_dir/Ppol_1.3.3.transcripts.mmseq.fasta"
F1_refseq="$refseq_dir/Pman_2.1.3_Ppol_1.3.3.renamed_suffix.diploid.transcripts.mmseq.fasta"

for tissue in $(cat ${list_dir}/tissuelist.txt)

do
    work_dir="$data_dir/${tissue}"                
    tissue_samplelist="$list_dir/${tissue}.samplelist.txt"
    
    (
    echo "#!/bin/bash"
    echo "#SBATCH -n 1"
    echo "#SBATCH -c 1"   
    echo "#SBATCH -N 1"
    echo "#SBATCH -t 0-02:00"
    echo "#SBATCH -p hoekstra,shared,general"
    echo "#SBATCH --mem=1gb"
    echo "#SBATCH --job-name bam2hits"
    echo "#SBATCH --array 1-18%18"
    echo "#SBATCH -o $scripts_dir/logs/%x_%A_%a.out"
    echo "#SBATCH -e $scripts_dir/logs/%x_%A_%a.err"
    echo ""
    printf "sample=\$(awk -v line=\$SLURM_ARRAY_TASK_ID 'NR==line {print \$1; exit}' $tissue_samplelist) \n" 
    echo ""
    printf "case \$sample in \n\
    BW*) refseq=\"$BW_refseq\" ;; \n\
    PO*) refseq=\"$PO_refseq\" ;; \n\
    F1*) refseq=\"$F1_refseq\" ;; \n\
    *) echo \"Sample name does not contain pop info. Refseq could not be specified.\" ;; \n\
    esac \n"
    echo ""
    printf "if [ ! -f $work_dir/\${sample}.bam2hits.hits ]; then \n \
    $mmseq_dir/bam2hits \
    \$refseq \
    $work_dir/\${sample}.STAR.Aligned.toTranscriptome.out.bam \
    > $work_dir/\${sample}.bam2hits.hits; fi \n"
    ) > ${scripts_dir}/bam2hits.${tissue}.slurm

done

sed -i 's/--array 1-18%18/--array 1-11%11/' $scripts_dir/bam2hits.wholebrain.slurm

In [ ]:
%%script env list_dir="$list_dir" scripts_dir="$scripts_dir" bash

for tissue in $(grep -w "thalamus" ${list_dir}/tissuelist.txt)

do 
    printf "$tissue -->\t" >> $scripts_dir/logs/jobIDs.bam2hits.log
    sbatch ${scripts_dir}/bam2hits.${tissue}.slurm >> $scripts_dir/logs/jobIDs.bam2hits.log
    sleep 1
done

cat $scripts_dir/logs/jobIDs.bam2hits.log

In [ ]:
%%script env list_dir="$list_dir" scripts_dir="$scripts_dir" bash

sacct -j $(grep -o '[0-9]*' $scripts_dir/logs/jobIDs.bam2hits.log | paste -s -d ",") | grep -v "COMP"

#### MMSEQ

In [ ]:
%%script env data_dir="$data_dir" scripts_dir="$scripts_dir" list_dir="$list_dir" mmseq_dir="$mmseq_dir" bash

for tissue in $(cat ${list_dir}/tissuelist.txt)

do
    work_dir="$data_dir/${tissue}"                
    tissue_samplelist="$list_dir/${tissue}.samplelist.txt"
    
    (
    echo "#!/bin/bash"
    echo "#SBATCH -n 1"
    echo "#SBATCH -c 1"   
    echo "#SBATCH -N 1"
    echo "#SBATCH -t 0-12:00"
    echo "#SBATCH -p hoekstra,shared,general"
    echo "#SBATCH --mem=6gb"
    echo "#SBATCH --job-name MMSEQ"
    echo "#SBATCH --array 1-18%18"
    echo "#SBATCH -o $scripts_dir/logs/%x_%A_%a.out"
    echo "#SBATCH -e $scripts_dir/logs/%x_%A_%a.err"
    echo ""
    printf "sample=\$(awk -v line=\$SLURM_ARRAY_TASK_ID 'NR==line {print \$1; exit}' $tissue_samplelist) \n" 
    echo ""
    printf "if [ ! -f $work_dir/\${sample}.MMSEQ.mmseq ]; then \n \
    $mmseq_dir/mmseq \
    $work_dir/\${sample}.bam2hits.hits \
    $work_dir/\${sample}.MMSEQ; fi \n"
    ) > ${scripts_dir}/MMSEQ.${tissue}.slurm

done

sed -i 's/--array 1-18%18/--array 1-11%11/' $scripts_dir/MMSEQ.wholebrain.slurm

In [ ]:
%%script env list_dir="$list_dir" scripts_dir="$scripts_dir" bash

for tissue in $(cat ${list_dir}/tissuelist.txt)

do 
    printf "$tissue -->\t" >> $scripts_dir/logs/jobIDs.MMSEQ.log
    sbatch ${scripts_dir}/MMSEQ.${tissue}.slurm >> $scripts_dir/logs/jobIDs.MMSEQ.log
    sleep 1
done

cat $scripts_dir/logs/jobIDs.MMSEQ.log

In [ ]:
%%script env list_dir="$list_dir" scripts_dir="$scripts_dir" bash

sacct -j $(grep -o '[0-9]*' $scripts_dir/logs/jobIDs.MMSEQ.log | paste -s -d ",") | grep -v "COMP"

#### Extract shared features in parental species

In [ ]:
%%script env data_dir="$data_dir" scripts_dir="$scripts_dir" list_dir="$list_dir" bash

for tissue in $(cat $list_dir/tissuelist.txt)
do
    for sample in $(grep "BW\|PO" $list_dir/${tissue}.samplelist.txt)
    do

    if [ ! -f $data_dir/$tissue/${sample}.MMSEQ.gene.shared.mmseq ]; then 
        awk 'BEGIN{OFS="\t"} NR<=2 {print} FNR==NR && !/APOLLO/ && !/None/ \
        {gsub("DEERMOUSE_G[0-9]+_|OLDFIELD_G[0-9]+_","",$1); idx[$1]=$0} \
        FNR!=NR {if ($1 in idx) print idx[$1]}' \
        $data_dir/$tissue/${sample}.MMSEQ.gene.mmseq \
        $list_dir/BW_PO.shared_genes_byName.genelist.txt \
        > $data_dir/$tissue/${sample}.MMSEQ.gene.shared.mmseq; fi
    done
done

#### Get counts for parental alleles in F1s separately

In [ ]:
%%script env data_dir="$data_dir" scripts_dir="$scripts_dir" list_dir="$list_dir" bash

for tissue in $(cat $list_dir/tissuelist.txt)
do
    for sample in $(grep "F1" $list_dir/${tissue}.samplelist.txt)
    do
        if [ ! -f $data_dir/$tissue/${sample}.MMSEQ.gene.shared.BW.mmseq ]; then
            awk 'BEGIN{OFS="\t"} NR<=2 {print} FNR==NR && /BW/ && !/APOLLO/ && !/None/ \
            {gsub("DEERMOUSE_G[0-9]+_|OLDFIELD_G[0-9]+_|_BW|_PO|BW_|PO_","",$1); \
            gsub("_BW|_PO|BW_|PO_","",$2); idx[$1]=$0} \
            FNR!=NR {if ($1 in idx) print idx[$1]}' \
            $data_dir/$tissue/${sample}.MMSEQ.gene.mmseq \
            $list_dir/BW_PO.shared_genes_byName.genelist.txt \
            > $data_dir/$tissue/${sample}.MMSEQ.gene.shared.BW.mmseq; fi
    
        if [ ! -f $data_dir/$tissue/${sample}.MMSEQ.gene.shared.PO.mmseq ]; then
            awk 'BEGIN{OFS="\t"} NR<=2 {print} FNR==NR && /PO/ && !/APOLLO/ && !/None/ \
            {gsub("DEERMOUSE_G[0-9]+_|OLDFIELD_G[0-9]+_|_BW|_PO|BW_|PO_","",$1); \
            gsub("_BW|_PO|BW_|PO_","",$2); idx[$1]=$0} \
            FNR!=NR {if ($1 in idx) print idx[$1]}' \
            $data_dir/$tissue/${sample}.MMSEQ.gene.mmseq \
            $list_dir/BW_PO.shared_genes_byName.genelist.txt \
            > $data_dir/$tissue/${sample}.MMSEQ.gene.shared.PO.mmseq; fi
    done
done

### Compare propability of models with MMDIFF

#### Create design matrix files

In [ ]:
%%script env list_dir="$list_dir" scripts_dir="$scripts_dir" data_dir="$data_dir" bash

if [ ! -d $data_dir/MMDIFF/design_matrices ]; then mkdir -p $data_dir/MMDIFF/design_matrices; fi

*NOTE*: One sample (F1_F_15462_hypothalamus) removed, because it was outlier in PCA. Manually changed design matrix files for controls (5 instead of 6 F1s).

##### Single tissue - cons vs. cis with sex as covariate

In [ ]:
%%script env list_dir="$list_dir" scripts_dir="$scripts_dir" data_dir="$data_dir" bash

for tissue in $(grep -v "wholebrain" $list_dir/tissuelist.txt)
do
    samples=""
    samples+=$(find $data_dir/$tissue -type f -name "BW*.MMSEQ.gene.shared.mmseq" -exec basename {} \;)
    samples+=" "
    samples+=$(find $data_dir/$tissue -type f -name "PO*.MMSEQ.gene.shared.mmseq" -exec basename {} \;)
    samples+=" "
    samples+=$(find $data_dir/$tissue -type f -name "F1*.MMSEQ.gene.shared.BW.mmseq" -exec basename {} \;)
    samples+=" "
    samples+=$(find $data_dir/$tissue -type f -name "F1*.MMSEQ.gene.shared.PO.mmseq" -exec basename {} \;)
    samples+=" "

    if [ ! -f $data_dir/MMDIFF/design_matrices/${tissue}.mmdiff.ASE.cons_vs_cis.covarSex.matrix.txt ]; then
        (echo $samples | tr " " "\n" | grep -v "F1_F_15462_hypothalamus" | \
        awk 'BEGIN{OFS=" "; print "# M; no. of rows = no. of observations"} \
        /_M_/ {print "0"; next} \
        /_F_/ {print "1"; next}'
        echo $samples | tr " " "\n" | grep -v "F1_F_15462_hypothalamus" | \
        awk 'BEGIN{OFS=" "; print "# C; no. of rows = no. of observations and no. of columns = 2 (one for each model)"} \
        (/BW/ && !/F1/) {print "0","0"; next} \
        (/PO/ && !/F1/) {print "1","1"; next} \
        (/F1/ && /BW/) {print "2","2"; next} \
        (/F1/ && /PO/) {print "3","3"; next}' 
        printf '# P0(collapsed); no. of rows = no. of classes for model 0\n'
        printf '%s\n' "0.5"
        printf '%s\n' "0.5"
        printf '%s\n' "-0.5"   
        printf '%s\n' "-0.5"    
        printf '# P1(collapsed); no. of rows = no. of classes for model 1\n'
        printf '%s %s\n' "0.5" "0.5"
        printf '%s %s\n' "0.5" "-0.5"
        printf '%s %s\n' "-0.5" "0.5"
        printf '%s %s\n' "-0.5" "-0.5") \
        > $data_dir/MMDIFF/design_matrices/${tissue}.mmdiff.ASE.cons_vs_cis.covarSex.matrix.txt; fi    
done

In [ ]:
%%script env list_dir="$list_dir" scripts_dir="$scripts_dir" data_dir="$data_dir" bash

### CONTROL --> assign 3 BWs to POs and vice versa (same in F1 alleles) 

number=24  

for tissue in $(grep -v "wholebrain" $list_dir/tissuelist.txt)
do
    samples=""
    samples+=$(find $data_dir/$tissue -type f -name "BW*.MMSEQ.gene.shared.mmseq" -exec basename {} \;)
    samples+=" "
    samples+=$(find $data_dir/$tissue -type f -name "PO*.MMSEQ.gene.shared.mmseq" -exec basename {} \;)
    samples+=" "
    samples+=$(find $data_dir/$tissue -type f -name "F1*.MMSEQ.gene.shared.BW.mmseq" -exec basename {} \;)
    samples+=" "
    samples+=$(find $data_dir/$tissue -type f -name "F1*.MMSEQ.gene.shared.PO.mmseq" -exec basename {} \;)
    samples+=" "
    
    if [ ! -f $data_dir/MMDIFF/design_matrices/${tissue}.mmdiff.ASE.cons_vs_cis.covarSex.control.matrix.txt ]; then
        (echo $samples | tr " " "\n" | grep -v "F1_F_15462_hypothalamus" | \
        awk 'BEGIN{OFS=" "; print "# M; no. of rows = no. of observations"} \
        /_M_/ {print "0"; next} \
        /_F_/ {print "1"; next}'
        printf '# C; no. of rows = no. of observations and no. of columns = 2 (one for each model)\n'
        printf '%.s0 0\n1 1\n' $(seq 1 $[$number/4])
        printf '%.s2 2\n3 3\n' $(seq 1 $[$number/4])    
        printf '# P0(collapsed); no. of rows = no. of classes for model 0\n'
        printf '%s\n' "0.5"
        printf '%s\n' "0.5"
        printf '%s\n' "-0.5"   
        printf '%s\n' "-0.5"    
        printf '# P1(collapsed); no. of rows = no. of classes for model 1\n'
        printf '%s %s\n' "0.5" "0.5"
        printf '%s %s\n' "0.5" "-0.5"
        printf '%s %s\n' "-0.5" "0.5"
        printf '%s %s\n' "-0.5" "-0.5") \
        > $data_dir/MMDIFF/design_matrices/${tissue}.mmdiff.ASE.cons_vs_cis.covarSex.control.matrix.txt; fi    
done

##### Single tissue - cons vs. trans with sex as covariate

In [ ]:
%%script env list_dir="$list_dir" scripts_dir="$scripts_dir" data_dir="$data_dir" bash

for tissue in $(grep -v "wholebrain" $list_dir/tissuelist.txt)
do
    samples=""
    samples+=$(find $data_dir/$tissue -type f -name "BW*.MMSEQ.gene.shared.mmseq" -exec basename {} \;)
    samples+=" "
    samples+=$(find $data_dir/$tissue -type f -name "PO*.MMSEQ.gene.shared.mmseq" -exec basename {} \;)
    samples+=" "
    samples+=$(find $data_dir/$tissue -type f -name "F1*.MMSEQ.gene.shared.BW.mmseq" -exec basename {} \;)
    samples+=" "
    samples+=$(find $data_dir/$tissue -type f -name "F1*.MMSEQ.gene.shared.PO.mmseq" -exec basename {} \;)
    samples+=" "

    if [ ! -f $data_dir/MMDIFF/design_matrices/${tissue}.mmdiff.ASE.cons_vs_trans.covarSex.matrix.txt ]; then
        (echo $samples | tr " " "\n" | grep -v "F1_F_15462_hypothalamus" | \
        awk 'BEGIN{OFS=" "; print "# M; no. of rows = no. of observations"} \
        /_M_/ {print "0"; next} \
        /_F_/ {print "1"; next}'
        echo $samples | tr " " "\n" | grep -v "F1_F_15462_hypothalamus" | \
        awk 'BEGIN{OFS=" "; print "# C; no. of rows = no. of observations and no. of columns = 2 (one for each model)"} \
        (/BW/ && !/F1/) {print "0","0"; next} \
        (/PO/ && !/F1/) {print "1","1"; next} \
        (/F1/ && /BW/) {print "2","2"; next} \
        (/F1/ && /PO/) {print "3","3"; next}' 
        printf '# P0(collapsed); no. of rows = no. of classes for model 0\n'
        printf '%s\n' "0.5"
        printf '%s\n' "0.5"
        printf '%s\n' "-0.5"   
        printf '%s\n' "-0.5"    
        printf '# P1(collapsed); no. of rows = no. of classes for model 1\n'
        printf '%s %s\n' "0.5" "0.5"
        printf '%s %s\n' "0.5" "-0.5"
        printf '%s %s\n' "-0.5" "0"
        printf '%s %s\n' "-0.5" "0") \
        > $data_dir/MMDIFF/design_matrices/${tissue}.mmdiff.ASE.cons_vs_trans.covarSex.matrix.txt; fi    
done

In [ ]:
%%script env list_dir="$list_dir" scripts_dir="$scripts_dir" data_dir="$data_dir" bash

### CONTROL --> assign 3 BWs to POs and vice versa (same in F1 alleles) 

number=24  

for tissue in $(grep -v "wholebrain" $list_dir/tissuelist.txt)
do
    samples=""
    samples+=$(find $data_dir/$tissue -type f -name "BW*.MMSEQ.gene.shared.mmseq" -exec basename {} \;)
    samples+=" "
    samples+=$(find $data_dir/$tissue -type f -name "PO*.MMSEQ.gene.shared.mmseq" -exec basename {} \;)
    samples+=" "
    samples+=$(find $data_dir/$tissue -type f -name "F1*.MMSEQ.gene.shared.BW.mmseq" -exec basename {} \;)
    samples+=" "
    samples+=$(find $data_dir/$tissue -type f -name "F1*.MMSEQ.gene.shared.PO.mmseq" -exec basename {} \;)
    samples+=" "
    
    if [ ! -f $data_dir/MMDIFF/design_matrices/${tissue}.mmdiff.ASE.cons_vs_trans.covarSex.control.matrix.txt ]; then
        (echo $samples | tr " " "\n" | grep -v "F1_F_15462_hypothalamus" | \
        awk 'BEGIN{OFS=" "; print "# M; no. of rows = no. of observations"} \
        /_M_/ {print "0"; next} \
        /_F_/ {print "1"; next}'
        printf '# C; no. of rows = no. of observations and no. of columns = 2 (one for each model)\n'
        printf '%.s0 0\n1 1\n' $(seq 1 $[$number/4])
        printf '%.s2 2\n3 3\n' $(seq 1 $[$number/4])    
        printf '# P0(collapsed); no. of rows = no. of classes for model 0\n'
        printf '%s\n' "0.5"
        printf '%s\n' "0.5"
        printf '%s\n' "-0.5"   
        printf '%s\n' "-0.5"    
        printf '# P1(collapsed); no. of rows = no. of classes for model 1\n'
        printf '%s %s\n' "0.5" "0.5"
        printf '%s %s\n' "0.5" "-0.5"
        printf '%s %s\n' "-0.5" "0"
        printf '%s %s\n' "-0.5" "0") \
        > $data_dir/MMDIFF/design_matrices/${tissue}.mmdiff.ASE.cons_vs_trans.covarSex.control.matrix.txt; fi    
done

##### Single tissue - cons vs. cistrans with sex as covariate

In [ ]:
%%script env list_dir="$list_dir" scripts_dir="$scripts_dir" data_dir="$data_dir" bash

for tissue in $(grep -v "wholebrain" $list_dir/tissuelist.txt)
do
    samples=""
    samples+=$(find $data_dir/$tissue -type f -name "BW*.MMSEQ.gene.shared.mmseq" -exec basename {} \;)
    samples+=" "
    samples+=$(find $data_dir/$tissue -type f -name "PO*.MMSEQ.gene.shared.mmseq" -exec basename {} \;)
    samples+=" "
    samples+=$(find $data_dir/$tissue -type f -name "F1*.MMSEQ.gene.shared.BW.mmseq" -exec basename {} \;)
    samples+=" "
    samples+=$(find $data_dir/$tissue -type f -name "F1*.MMSEQ.gene.shared.PO.mmseq" -exec basename {} \;)
    samples+=" "

    if [ ! -f $data_dir/MMDIFF/design_matrices/${tissue}.mmdiff.ASE.cons_vs_cistrans.covarSex.matrix.txt ]; then
        (echo $samples | tr " " "\n" | grep -v "F1_F_15462_hypothalamus" | \
        awk 'BEGIN{OFS=" "; print "# M; no. of rows = no. of observations"} \
        /_M_/ {print "0"; next} \
        /_F_/ {print "1"; next}'
        echo $samples | tr " " "\n" | grep -v "F1_F_15462_hypothalamus" | \
        awk 'BEGIN{OFS=" "; print "# C; no. of rows = no. of observations and no. of columns = 2 (one for each model)"} \
        (/BW/ && !/F1/) {print "0","0"; next} \
        (/PO/ && !/F1/) {print "1","1"; next} \
        (/F1/ && /BW/) {print "2","2"; next} \
        (/F1/ && /PO/) {print "3","3"; next}' 
        printf '# P0(collapsed); no. of rows = no. of classes for model 0\n'
        printf '%s\n' "0.5"
        printf '%s\n' "0.5"
        printf '%s\n' "-0.5"   
        printf '%s\n' "-0.5"    
        printf '# P1(collapsed); no. of rows = no. of classes for model 1\n'
        printf '%s %s %s\n' "0.5" "0.5 ""0"
        printf '%s %s %s\n' "0.5" "-0.5" "0"
        printf '%s %s %s\n' "-0.5" "0" "0.5"
        printf '%s %s %s\n' "-0.5" "0" "-0.5") \
        > $data_dir/MMDIFF/design_matrices/${tissue}.mmdiff.ASE.cons_vs_cistrans.covarSex.matrix.txt; fi    
done

In [ ]:
%%script env list_dir="$list_dir" scripts_dir="$scripts_dir" data_dir="$data_dir" bash

### CONTROL --> assign 3 BWs to POs and vice versa (same in F1 alleles) 

number=24  

for tissue in $(grep -v "wholebrain" $list_dir/tissuelist.txt)
do
    samples=""
    samples+=$(find $data_dir/$tissue -type f -name "BW*.MMSEQ.gene.shared.mmseq" -exec basename {} \;)
    samples+=" "
    samples+=$(find $data_dir/$tissue -type f -name "PO*.MMSEQ.gene.shared.mmseq" -exec basename {} \;)
    samples+=" "
    samples+=$(find $data_dir/$tissue -type f -name "F1*.MMSEQ.gene.shared.BW.mmseq" -exec basename {} \;)
    samples+=" "
    samples+=$(find $data_dir/$tissue -type f -name "F1*.MMSEQ.gene.shared.PO.mmseq" -exec basename {} \;)
    samples+=" "
    
    if [ ! -f $data_dir/MMDIFF/design_matrices/${tissue}.mmdiff.ASE.cons_vs_cistrans.covarSex.control.matrix.txt ]; then
        (echo $samples | tr " " "\n" | grep -v "F1_F_15462_hypothalamus" | \
        awk 'BEGIN{OFS=" "; print "# M; no. of rows = no. of observations"} \
        /_M_/ {print "0"; next} \
        /_F_/ {print "1"; next}'
        printf '# C; no. of rows = no. of observations and no. of columns = 2 (one for each model)\n'
        printf '%.s0 0\n1 1\n' $(seq 1 $[$number/4])
        printf '%.s2 2\n3 3\n' $(seq 1 $[$number/4])    
        printf '# P0(collapsed); no. of rows = no. of classes for model 0\n'
        printf '%s\n' "0.5"
        printf '%s\n' "0.5"
        printf '%s\n' "-0.5"   
        printf '%s\n' "-0.5"    
        printf '# P1(collapsed); no. of rows = no. of classes for model 1\n'
        printf '%s %s %s\n' "0.5" "0.5 ""0"
        printf '%s %s %s\n' "0.5" "-0.5" "0"
        printf '%s %s %s\n' "-0.5" "0" "0.5"
        printf '%s %s %s\n' "-0.5" "0" "-0.5") \
        > $data_dir/MMDIFF/design_matrices/${tissue}.mmdiff.ASE.cons_vs_cistrans.covarSex.control.matrix.txt; fi    
done

##### Single tissue - cons vs. comp with sex as covariate

In [ ]:
%%script env list_dir="$list_dir" scripts_dir="$scripts_dir" data_dir="$data_dir" bash

for tissue in $(grep -v "wholebrain" $list_dir/tissuelist.txt)
do
    samples=""
    samples+=$(find $data_dir/$tissue -type f -name "BW*.MMSEQ.gene.shared.mmseq" -exec basename {} \;)
    samples+=" "
    samples+=$(find $data_dir/$tissue -type f -name "PO*.MMSEQ.gene.shared.mmseq" -exec basename {} \;)
    samples+=" "
    samples+=$(find $data_dir/$tissue -type f -name "F1*.MMSEQ.gene.shared.BW.mmseq" -exec basename {} \;)
    samples+=" "
    samples+=$(find $data_dir/$tissue -type f -name "F1*.MMSEQ.gene.shared.PO.mmseq" -exec basename {} \;)
    samples+=" "

    if [ ! -f $data_dir/MMDIFF/design_matrices/${tissue}.mmdiff.ASE.cons_vs_comp.covarSex.matrix.txt ]; then
        (echo $samples | tr " " "\n" | grep -v "F1_F_15462_hypothalamus" | \
        awk 'BEGIN{OFS=" "; print "# M; no. of rows = no. of observations"} \
        /_M_/ {print "0"; next} \
        /_F_/ {print "1"; next}'
        echo $samples | tr " " "\n" | grep -v "F1_F_15462_hypothalamus" | \
        awk 'BEGIN{OFS=" "; print "# C; no. of rows = no. of observations and no. of columns = 2 (one for each model)"} \
        (/BW/ && !/F1/) {print "0","0"; next} \
        (/PO/ && !/F1/) {print "1","1"; next} \
        (/F1/ && /BW/) {print "2","2"; next} \
        (/F1/ && /PO/) {print "3","3"; next}' 
        printf '# P0(collapsed); no. of rows = no. of classes for model 0\n'
        printf '%s\n' "0.5"
        printf '%s\n' "0.5"
        printf '%s\n' "-0.5"   
        printf '%s\n' "-0.5"    
        printf '# P1(collapsed); no. of rows = no. of classes for model 1\n'
        printf '%s %s\n' "0.5" "0"
        printf '%s %s\n' "0.5" "0"
        printf '%s %s\n' "-0.5" "0.5"
        printf '%s %s\n' "-0.5" "-0.5") \
        > $data_dir/MMDIFF/design_matrices/${tissue}.mmdiff.ASE.cons_vs_comp.covarSex.matrix.txt; fi    
done

In [ ]:
%%script env list_dir="$list_dir" scripts_dir="$scripts_dir" data_dir="$data_dir" bash

### CONTROL --> assign 3 BWs to POs and vice versa (same in F1 alleles) 

number=24  

for tissue in $(grep -v "wholebrain" $list_dir/tissuelist.txt)
do
    samples=""
    samples+=$(find $data_dir/$tissue -type f -name "BW*.MMSEQ.gene.shared.mmseq" -exec basename {} \;)
    samples+=" "
    samples+=$(find $data_dir/$tissue -type f -name "PO*.MMSEQ.gene.shared.mmseq" -exec basename {} \;)
    samples+=" "
    samples+=$(find $data_dir/$tissue -type f -name "F1*.MMSEQ.gene.shared.BW.mmseq" -exec basename {} \;)
    samples+=" "
    samples+=$(find $data_dir/$tissue -type f -name "F1*.MMSEQ.gene.shared.PO.mmseq" -exec basename {} \;)
    samples+=" "
    
    if [ ! -f $data_dir/MMDIFF/design_matrices/${tissue}.mmdiff.ASE.cons_vs_comp.covarSex.control.matrix.txt ]; then
        (echo $samples | tr " " "\n" | grep -v "F1_F_15462_hypothalamus" | \
        awk 'BEGIN{OFS=" "; print "# M; no. of rows = no. of observations"} \
        /_M_/ {print "0"; next} \
        /_F_/ {print "1"; next}'
        printf '# C; no. of rows = no. of observations and no. of columns = 2 (one for each model)\n'
        printf '%.s0 0\n1 1\n' $(seq 1 $[$number/4])
        printf '%.s2 2\n3 3\n' $(seq 1 $[$number/4])    
        printf '# P0(collapsed); no. of rows = no. of classes for model 0\n'
        printf '%s\n' "0.5"
        printf '%s\n' "0.5"
        printf '%s\n' "-0.5"   
        printf '%s\n' "-0.5"    
        printf '# P1(collapsed); no. of rows = no. of classes for model 1\n'
        printf '%s %s\n' "0.5" "0"
        printf '%s %s\n' "0.5" "0"
        printf '%s %s\n' "-0.5" "0.5"
        printf '%s %s\n' "-0.5" "-0.5") \
        > $data_dir/MMDIFF/design_matrices/${tissue}.mmdiff.ASE.cons_vs_comp.covarSex.control.matrix.txt; fi    
done

#### Run MMDIFF

##### Single tissue with sex as covariate

In [ ]:
%%script env scripts_dir="$scripts_dir" list_dir="$list_dir" data_dir="$data_dir" mmseq_dir="$mmseq_dir" bash

models="cons_vs_cis cons_vs_trans cons_vs_cistrans cons_vs_comp"

for model in $models
do

    for tissue in $(grep -v "wholebrain" $list_dir/tissuelist.txt)
    do
        samples=""
        samples+=$(find $data_dir/$tissue -type f -name "BW*.MMSEQ.gene.shared.mmseq")
        samples+=" "
        samples+=$(find $data_dir/$tissue -type f -name "PO*.MMSEQ.gene.shared.mmseq")
        samples+=" "
        samples+=$(find $data_dir/$tissue -type f -name "F1*.MMSEQ.gene.shared.BW.mmseq" | grep -v "F1_F_15462_hypothalamus")
        samples+=" "
        samples+=$(find $data_dir/$tissue -type f -name "F1*.MMSEQ.gene.shared.PO.mmseq" | grep -v "F1_F_15462_hypothalamus")
        samples+=" "
        
        if [ ! -f $data_dir/MMDIFF/out_mmdiff.${tissue}.ASE.${model}.covarSex.seed123.out ]; then
            printf "submitting job for %s %s:\t" $tissue $model
            sbatch -n 1 -c 8 --contiguous --mem=500mb --time=00-12:00 -p hoekstra,shared,serial_requeue --job-name mmdiff \
            -e $data_dir/MMDIFF/out_mmdiff.${tissue}.ASE.${model}.covarSex.seed123.log \
            -o $data_dir/MMDIFF/out_mmdiff.${tissue}.ASE.${model}.covarSex.seed123.stdout \
            --wrap="OMP_NUM_THREADS=8 $mmseq_dir/mmdiff \
            -burnin 10240 -iter 25600 -seed 123 \
            -m $data_dir/MMDIFF/design_matrices/${tissue}.mmdiff.ASE.${model}.covarSex.matrix.txt \
            $(echo $samples) \
            > $data_dir/MMDIFF/out_mmdiff.${tissue}.ASE.${model}.covarSex.seed123.out"
            sleep 1; fi        
    done
done

In [ ]:
%%script env scripts_dir="$scripts_dir" list_dir="$list_dir" data_dir="$data_dir" mmseq_dir="$mmseq_dir" bash

### CONTROLS

models="cons_vs_cis cons_vs_trans cons_vs_cistrans cons_vs_comp"

for model in $models
do

    for tissue in $(grep -v "wholebrain" $list_dir/tissuelist.txt)
    do
        samples=""
        samples+=$(find $data_dir/$tissue -type f -name "BW*.MMSEQ.gene.shared.mmseq")
        samples+=" "
        samples+=$(find $data_dir/$tissue -type f -name "PO*.MMSEQ.gene.shared.mmseq")
        samples+=" "
        samples+=$(find $data_dir/$tissue -type f -name "F1*.MMSEQ.gene.shared.BW.mmseq" | grep -v "F1_F_15462_hypothalamus")
        samples+=" "
        samples+=$(find $data_dir/$tissue -type f -name "F1*.MMSEQ.gene.shared.PO.mmseq" | grep -v "F1_F_15462_hypothalamus")
        samples+=" "
        
        if [ ! -f $data_dir/MMDIFF/out_mmdiff.${tissue}.ASE.${model}.covarSex.control.out ]; then
            printf "submitting job for %s %s:\t" $tissue $model
            sbatch -n 1 -c 8 --contiguous --mem=500mb --time=00-12:00 -p hoekstra,shared,serial_requeue --job-name mmdiff \
            -e $data_dir/MMDIFF/out_mmdiff.${tissue}.ASE.${model}.covarSex.control.err \
            -o $data_dir/MMDIFF/out_mmdiff.${tissue}.ASE.${model}.covarSex.control.log \
            --wrap="OMP_NUM_THREADS=8 $mmseq_dir/mmdiff \
            -burnin 10240 -iter 25600 -seed 1234 \
            -m $data_dir/MMDIFF/design_matrices/${tissue}.mmdiff.ASE.${model}.covarSex.control.matrix.txt \
            $(echo $samples) \
            > $data_dir/MMDIFF/out_mmdiff.${tissue}.ASE.${model}.covarSex.control.out"
            sleep 1; fi        
    done
done

##### Bootstrapping

In [ ]:
%%script env scripts_dir="$scripts_dir" list_dir="$list_dir" data_dir="$data_dir" mmseq_dir="$mmseq_dir" bash

if [ ! -d $data_dir/MMDIFF/bootstrapping ]; then mkdir $data_dir/MMDIFF/bootstrapping; fi

tissues="cerebellum mPFC"

for tissue in $tissues
do

    for rep in {001..010}
    do
        
        if [ ! -d $data_dir/MMDIFF/bootstrapping/BSrep_${rep} ]; then mkdir $data_dir/MMDIFF/bootstrapping/BSrep_${rep}; fi
    
        samples=""
        samples+=$(find $data_dir/$tissue -type f -name "BW*.MMSEQ.gene.shared.mmseq" \
        | paste -s  -d " " | awk '{print $'"$((1+RANDOM%6))"',$'"$((1+RANDOM%6))"',$'"$((1+RANDOM%6))"',\
        $'"$((1+RANDOM%6))"',$'"$((1+RANDOM%6))"',$'"$((1+RANDOM%6))"'}')
        samples+=" "
        samples+=$(find $data_dir/$tissue -type f -name "PO*.MMSEQ.gene.shared.mmseq" \
        | paste -s  -d " " | awk '{print $'"$((1+RANDOM%6))"',$'"$((1+RANDOM%6))"',$'"$((1+RANDOM%6))"',\
        $'"$((1+RANDOM%6))"',$'"$((1+RANDOM%6))"',$'"$((1+RANDOM%6))"'}')
        samples+=" "
        samples+=$(find $data_dir/$tissue -type f -name "F1*.MMSEQ.gene.shared.BW.mmseq" \
        | paste -s  -d " " | awk '{print $'"$((1+RANDOM%6))"',$'"$((1+RANDOM%6))"',$'"$((1+RANDOM%6))"',\
        $'"$((1+RANDOM%6))"',$'"$((1+RANDOM%6))"',$'"$((1+RANDOM%6))"'}')
        samples+=" "
        samples+=$(find $data_dir/$tissue -type f -name "F1*.MMSEQ.gene.shared.PO.mmseq" \
        | paste -s  -d " " | awk '{print $'"$((1+RANDOM%6))"',$'"$((1+RANDOM%6))"',$'"$((1+RANDOM%6))"',\
        $'"$((1+RANDOM%6))"',$'"$((1+RANDOM%6))"',$'"$((1+RANDOM%6))"'}')
 
        model="cons_vs_cis"
    
        if [ ! -f $data_dir/MMDIFF/bootstrapping/BSrep_${rep}/${tissue}.mmdiff.ASE.${model}.covarSex.matrix.BSrep_${rep}.txt ]; then
            (echo $samples | sed "s,/n/scratchlfs/hoekstra_lab/akautt/projects/Peromyscus_RNAseq_brain/data/${tissue}/,,g" | tr " " "\n" | \
            awk 'BEGIN{OFS=" "; print "# M; no. of rows = no. of observations"} \
            /_M_/ {print "0"; next} \
            /_F_/ {print "1"; next}'
            echo $samples | sed "s,/n/scratchlfs/hoekstra_lab/akautt/projects/Peromyscus_RNAseq_brain/data/${tissue}/,,g" | tr " " "\n" | \
            awk 'BEGIN{OFS=" "; print "# C; no. of rows = no. of observations and no. of columns = 2 (one for each model)"} \
            (/BW/ && !/F1/) {print "0","0"; next} \
            (/PO/ && !/F1/) {print "1","1"; next} \
            (/F1/ && /BW/) {print "2","2"; next} \
            (/F1/ && /PO/) {print "3","3"; next}' 
            printf '# P0(collapsed); no. of rows = no. of classes for model 0\n'
            printf '%s\n' "0.5"
            printf '%s\n' "0.5"
            printf '%s\n' "-0.5"   
            printf '%s\n' "-0.5"    
            printf '# P1(collapsed); no. of rows = no. of classes for model 1\n'
            printf '%s %s\n' "0.5" "0.5"
            printf '%s %s\n' "0.5" "-0.5"
            printf '%s %s\n' "-0.5" "0.5"
            printf '%s %s\n' "-0.5" "-0.5"
            ) > $data_dir/MMDIFF/bootstrapping/BSrep_${rep}/${tissue}.mmdiff.ASE.${model}.covarSex.matrix.BSrep_${rep}.txt; fi    

        model="cons_vs_trans"
    
        if [ ! -f $data_dir/MMDIFF/bootstrapping/BSrep_${rep}/${tissue}.mmdiff.ASE.${model}.covarSex.matrix.BSrep_${rep}.txt ]; then
            (echo $samples | sed "s,/n/scratchlfs/hoekstra_lab/akautt/projects/Peromyscus_RNAseq_brain/data/${tissue}/,,g" | tr " " "\n" | \
            awk 'BEGIN{OFS=" "; print "# M; no. of rows = no. of observations"} \
            /_M_/ {print "0"; next} \
            /_F_/ {print "1"; next}'
            echo $samples | sed "s,/n/scratchlfs/hoekstra_lab/akautt/projects/Peromyscus_RNAseq_brain/data/${tissue}/,,g" | tr " " "\n" | \
            awk 'BEGIN{OFS=" "; print "# C; no. of rows = no. of observations and no. of columns = 2 (one for each model)"} \
            (/BW/ && !/F1/) {print "0","0"; next} \
            (/PO/ && !/F1/) {print "1","1"; next} \
            (/F1/ && /BW/) {print "2","2"; next} \
            (/F1/ && /PO/) {print "3","3"; next}' 
            printf '# P0(collapsed); no. of rows = no. of classes for model 0\n'
            printf '%s\n' "0.5"
            printf '%s\n' "0.5"
            printf '%s\n' "-0.5"   
            printf '%s\n' "-0.5"    
            printf '# P1(collapsed); no. of rows = no. of classes for model 1\n'
            printf '%s %s\n' "0.5" "0.5"
            printf '%s %s\n' "0.5" "-0.5"
            printf '%s %s\n' "-0.5" "0"
            printf '%s %s\n' "-0.5" "0"
            ) > $data_dir/MMDIFF/bootstrapping/BSrep_${rep}/${tissue}.mmdiff.ASE.${model}.covarSex.matrix.BSrep_${rep}.txt; fi    

        model="cons_vs_cistrans"
    
        if [ ! -f $data_dir/MMDIFF/bootstrapping/BSrep_${rep}/${tissue}.mmdiff.ASE.${model}.covarSex.matrix.BSrep_${rep}.txt ]; then
            (echo $samples | sed "s,/n/scratchlfs/hoekstra_lab/akautt/projects/Peromyscus_RNAseq_brain/data/${tissue}/,,g" | tr " " "\n" | \
            awk 'BEGIN{OFS=" "; print "# M; no. of rows = no. of observations"} \
            /_M_/ {print "0"; next} \
            /_F_/ {print "1"; next}'
            echo $samples | sed "s,/n/scratchlfs/hoekstra_lab/akautt/projects/Peromyscus_RNAseq_brain/data/${tissue}/,,g" | tr " " "\n" | \
            awk 'BEGIN{OFS=" "; print "# C; no. of rows = no. of observations and no. of columns = 2 (one for each model)"} \
            (/BW/ && !/F1/) {print "0","0"; next} \
            (/PO/ && !/F1/) {print "1","1"; next} \
            (/F1/ && /BW/) {print "2","2"; next} \
            (/F1/ && /PO/) {print "3","3"; next}' 
            printf '# P0(collapsed); no. of rows = no. of classes for model 0\n'
            printf '%s\n' "0.5"
            printf '%s\n' "0.5"
            printf '%s\n' "-0.5"   
            printf '%s\n' "-0.5"    
            printf '# P1(collapsed); no. of rows = no. of classes for model 1\n'
            printf '%s %s %s\n' "0.5" "0.5" "0"
            printf '%s %s %s\n' "0.5" "-0.5" "0"
            printf '%s %s %s\n' "-0.5" "0" "0.5"
            printf '%s %s %s\n' "-0.5" "0" "-0.5"
            ) > $data_dir/MMDIFF/bootstrapping/BSrep_${rep}/${tissue}.mmdiff.ASE.${model}.covarSex.matrix.BSrep_${rep}.txt; fi    

        model="cons_vs_comp"
    
        if [ ! -f $data_dir/MMDIFF/bootstrapping/BSrep_${rep}/${tissue}.mmdiff.ASE.${model}.covarSex.matrix.BSrep_${rep}.txt ]; then
            (echo $samples | sed "s,/n/scratchlfs/hoekstra_lab/akautt/projects/Peromyscus_RNAseq_brain/data/${tissue}/,,g" | tr " " "\n" | \
            awk 'BEGIN{OFS=" "; print "# M; no. of rows = no. of observations"} \
            /_M_/ {print "0"; next} \
            /_F_/ {print "1"; next}'
            echo $samples | sed "s,/n/scratchlfs/hoekstra_lab/akautt/projects/Peromyscus_RNAseq_brain/data/${tissue}/,,g" | tr " " "\n" | \
            awk 'BEGIN{OFS=" "; print "# C; no. of rows = no. of observations and no. of columns = 2 (one for each model)"} \
            (/BW/ && !/F1/) {print "0","0"; next} \
            (/PO/ && !/F1/) {print "1","1"; next} \
            (/F1/ && /BW/) {print "2","2"; next} \
            (/F1/ && /PO/) {print "3","3"; next}' 
            printf '# P0(collapsed); no. of rows = no. of classes for model 0\n'
            printf '%s\n' "0.5"
            printf '%s\n' "0.5"
            printf '%s\n' "-0.5"   
            printf '%s\n' "-0.5"    
            printf '# P1(collapsed); no. of rows = no. of classes for model 1\n'
            printf '%s %s\n' "0.5" "0"
            printf '%s %s\n' "0.5" "0"
            printf '%s %s\n' "-0.5" "0.5"
            printf '%s %s\n' "-0.5" "-0.5"
            ) > $data_dir/MMDIFF/bootstrapping/BSrep_${rep}/${tissue}.mmdiff.ASE.${model}.covarSex.matrix.BSrep_${rep}.txt; fi    
                        
        
        for model in $(echo "cons_vs_cis cons_vs_trans cons_vs_cistrans cons_vs_comp")
    
        do    
            if [ ! -f $data_dir/MMDIFF/bootstrapping/BSrep_${rep}/out_mmdiff.${tissue}.ASE.${model}.covarSex.BSrep_${rep}.out ]; then
                printf "$tissue - $model - bootstrap replicate $rep: "
                sbatch -n 1 -c 8 --contiguous --mem=500mb --time=00-06:00 -p hoekstra,shared,general --job-name mmdiff \
                -e $data_dir/MMDIFF/bootstrapping/BSrep_${rep}/out_mmdiff.${tissue}.ASE.${model}.covarSex.BSrep_${rep}.log \
                -o $data_dir/MMDIFF/bootstrapping/BSrep_${rep}/out_mmdiff.${tissue}.ASE.${model}.covarSex.BSrep_${rep}.stdout \
                --wrap="OMP_NUM_THREADS=8 $mmseq_dir/mmdiff \
                -burnin 10240 -iter 25600 \
                -m $data_dir/MMDIFF/bootstrapping/BSrep_${rep}/${tissue}.mmdiff.ASE.${model}.covarSex.matrix.BSrep_${rep}.txt \
                $(echo $samples) \
                > $data_dir/MMDIFF/bootstrapping/BSrep_${rep}/out_mmdiff.${tissue}.ASE.${model}.covarSex.BSrep_${rep}.out"
                sleep 1; fi
        done        
    done
done

##### Polytomous model comparison

In [ ]:
%load_ext rpy2.ipython

In [ ]:
%%R -i data_dir -o data_dir

source("/n/home00/akautt/software/mmseq-latest/src/R/mmseq.R")

covar=".covarSex"
control=".control"  # ".control" | ""

tissues=c("amygdala","cerebellum","hindbrain","hippocampus","hypothalamus","midbrain","mPFC",
          "septum","striatum","thalamus")

for (tissue in tissues){ 

    setwd(file.path(paste(data_dir,"MMDIFF",sep="/")))
    
    ### flat prior
    
    if(!file.exists(paste("out_mmdiff.",tissue,".ASE",covar,control,".prior_flat0.2.poly.out", sep=""))){
    
        model_comp <- polyclass(files=c(
                          paste("out_mmdiff.",tissue,".ASE.cons_vs_cis",covar,control,".out",sep=""),
                          paste("out_mmdiff.",tissue,".ASE.cons_vs_trans",covar,control,".out",sep=""),
                          paste("out_mmdiff.",tissue,".ASE.cons_vs_cistrans",covar,control,".out",sep=""),
                          paste("out_mmdiff.",tissue,".ASE.cons_vs_comp",covar,control,".out",sep="")),
                          prior=c(0.2,0.2,0.2,0.2,0.2))
        
        write.table(model_comp, file=paste("out_mmdiff.",tissue,".ASE",covar,control,".prior_flat0.2.poly.out", sep=""), 
                    quote=F, col.names=T, row.names=F, sep="\t")
    }
}

In [ ]:
%%R -i data_dir -o data_dir

### for bootstrap replicates

source("~/software/mmseq-latest/src/R/mmseq.R")

covar=".covarSex"

tissues=c("cerebellum","mPFC")

for (tissue in tissues){ 

    for (BSrep in sprintf('%0.3d', 1:10)){
    
        setwd(file.path(paste(data_dir,"/MMDIFF","/bootstrapping","/BSrep_",BSrep, sep="")))
        
        #### flat prior
    
        if(!file.exists(paste("out_mmdiff.",tissue,".ASE",covar,".BSrep_",BSrep,".prior_flat0.2.poly.out", sep=""))){
            
            model_comp <- polyclass(files=c(
                      paste("out_mmdiff.",tissue,".ASE.cons_vs_cis",covar,".BSrep_",BSrep,".out",sep=""),
                      paste("out_mmdiff.",tissue,".ASE.cons_vs_trans",covar,".BSrep_",BSrep,".out",sep=""),
                      paste("out_mmdiff.",tissue,".ASE.cons_vs_cistrans",covar,".BSrep_",BSrep,".out",sep=""),
                      paste("out_mmdiff.",tissue,".ASE.cons_vs_comp",covar,".BSrep_",BSrep,".out",sep="")),
                      prior=c(0.2,0.2,0.2,0.2,0.2))
            
            write.table(model_comp, file=paste("out_mmdiff.",tissue,".ASE",covar,".BSrep_",BSrep,".prior_flat0.2.poly.out", sep=""), 
                        quote=F, col.names=T, row.names=F, sep="\t")
        }
    }
}